In [ ]:
!pip install pywhatkit

In [1]:
# Configuration
contacts = [
    "+919821789147",  # Replace with actual phone numbers
    "+919818979508",
]

message = """Hello, Testing Kargos"""


In [ ]:
import requests
import json
import time
from datetime import datetime

# ============================================================
# CONFIGURATION - Update these with your credentials
# ============================================================
PHONE_NUMBER_ID = "850720011454978"  # From Meta Business
ACCESS_TOKEN = "EAAUpEZC6VFNkBP4CWFZAxoZATXATORyAZCPPi8LW07amVNNuqqvVJl9uxZBgFQgwRhK5v8XqJlZCV61nOfjzSHbZCd7DWBV6bKfKUiba4ju0w3WZAmJbyo1ug3goFGmoCKAD25EWWGNWCSJ1qWLX6bDyWZAm8VtHSYnhCG1GNJpksTRZC080MKhM2QETyUNS4JWfErlsnGQqsZBwOCpnK6MZAdmHg7Kof0wbqWNff05ZCFJ1BZBggZD"  # From Meta Business
VERSION = "v21.0"  # API version

# WhatsApp API endpoint
API_URL = f"https://graph.facebook.com/{VERSION}/{PHONE_NUMBER_ID}/messages"

# Message delay (seconds) - to avoid rate limiting
DELAY_BETWEEN_MESSAGES = 2


class WhatsAppBroadcaster:
    """WhatsApp Business API Broadcaster"""
    
    def __init__(self, phone_number_id, access_token):
        self.phone_number_id = phone_number_id
        self.access_token = access_token
        self.api_url = f"https://graph.facebook.com/{VERSION}/{phone_number_id}/messages"
        self.headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }
    
    def send_text_message(self, to_phone, message_text):
        """
        Send a text message
        
        Args:
            to_phone: Phone number with country code (without +)
            message_text: Message content
        
        Returns:
            dict: API response or error
        """
        payload = {
            "messaging_product": "whatsapp",
            "recipient_type": "individual",
            "to": to_phone,
            "type": "text",
            "text": {
                "preview_url": True,  # Enable link previews
                "body": message_text
            }
        }
        
        try:
            response = requests.post(self.api_url, headers=self.headers, json=payload)
            response.raise_for_status()
            return {"success": True, "data": response.json()}
        except requests.exceptions.HTTPError as e:
            error_data = response.json() if response.text else {}
            return {"success": False, "error": error_data}
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    def send_template_message(self, to_phone, template_name, language_code="en"):
        """
        Send a pre-approved template message
        
        Args:
            to_phone: Phone number with country code
            template_name: Name of approved template
            language_code: Language code (default: en)
        
        Returns:
            dict: API response
        """
        payload = {
            "messaging_product": "whatsapp",
            "to": to_phone,
            "type": "template",
            "template": {
                "name": template_name,
                "language": {
                    "code": language_code
                }
            }
        }
        
        try:
            response = requests.post(self.api_url, headers=self.headers, json=payload)
            response.raise_for_status()
            return {"success": True, "data": response.json()}
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    def send_broadcast(self, contacts, message, use_template=False, template_name=None):
        """
        Send broadcast message to multiple contacts
        
        Args:
            contacts: List of phone numbers (without +)
            message: Message text or template name
            use_template: Boolean - use template message
            template_name: Template name if use_template is True
        
        Returns:
            dict: Broadcast results
        """
        print(f"\n{'='*60}")
        print(f"WhatsApp Broadcast Started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*60}")
        print(f"Total contacts: {len(contacts)}")
        print(f"Message type: {'Template' if use_template else 'Text'}")
        print(f"{'='*60}\n")
        
        results = {
            "total": len(contacts),
            "success": 0,
            "failed": 0,
            "details": [],
            "timestamp": datetime.now().isoformat()
        }
        
        for idx, phone in enumerate(contacts, 1):
            print(f"[{idx}/{len(contacts)}] Sending to +{phone}...", end=" ")
            
            if use_template and template_name:
                response = self.send_template_message(phone, template_name)
            else:
                response = self.send_text_message(phone, message)
            
            if response["success"]:
                message_id = response["data"]["messages"][0]["id"]
                print(f"✓ Success (ID: {message_id})")
                results["success"] += 1
                results["details"].append({
                    "phone": f"+{phone}",
                    "status": "success",
                    "message_id": message_id,
                    "timestamp": datetime.now().isoformat()
                })
            else:
                error_msg = response.get("error", "Unknown error")
                print(f"✗ Failed - {error_msg}")
                results["failed"] += 1
                results["details"].append({
                    "phone": f"+{phone}",
                    "status": "failed",
                    "error": error_msg,
                    "timestamp": datetime.now().isoformat()
                })
            
            # Wait before next message (except for last one)
            if idx < len(contacts):
                time.sleep(DELAY_BETWEEN_MESSAGES)
        
        # Print summary
        self._print_summary(results)
        
        return results
    
    def _print_summary(self, results):
        """Print broadcast summary"""
        print(f"\n{'='*60}")
        print("BROADCAST SUMMARY")
        print(f"{'='*60}")
        print(f"Total contacts: {results['total']}")
        print(f"✓ Successful: {results['success']} ({results['success']/results['total']*100:.1f}%)")
        print(f"✗ Failed: {results['failed']} ({results['failed']/results['total']*100:.1f}%)")
        print(f"Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*60}\n")
    
    def verify_credentials(self):
        """Verify API credentials are working"""
        test_url = f"https://graph.facebook.com/{VERSION}/{self.phone_number_id}"
        try:
            response = requests.get(test_url, headers=self.headers)
            response.raise_for_status()
            print("✓ Credentials verified successfully!")
            print(f"Phone Number: {response.json().get('display_phone_number', 'N/A')}")
            return True
        except Exception as e:
            print(f"✗ Credential verification failed: {e}")
            return False


def save_results_to_file(results, filename="broadcast_results.json"):
    """Save broadcast results to JSON file"""
    with open(filename, "w") as f:
        json.dump(results, f, indent=2)
    print(f"Results saved to {filename}")


# ============================================================
# MAIN EXECUTION
# ============================================================
if __name__ == "__main__":
    print("\n" + "="*60)
    print("WhatsApp Business API - Broadcast Script")
    print("="*60 + "\n")
    
    # Check if credentials are configured
    if PHONE_NUMBER_ID == "your_phone_number_id" or ACCESS_TOKEN == "your_access_token":
        print("⚠️  ERROR: Please configure your credentials first!\n")
        print("Steps to get credentials:")
        print("1. Go to https://developers.facebook.com")
        print("2. Create an app and add WhatsApp product")
        print("3. Go to WhatsApp > API Setup")
        print("4. Copy 'Phone number ID' and 'Access token'")
        print("5. Update PHONE_NUMBER_ID and ACCESS_TOKEN in this script\n")
        exit(1)
    
    # Initialize broadcaster
    broadcaster = WhatsAppBroadcaster(PHONE_NUMBER_ID, ACCESS_TOKEN)
    
    # Verify credentials
    print("Verifying credentials...")
    if not broadcaster.verify_credentials():
        exit(1)
    
    print("\n")
    
    # ============================================================
    # CONFIGURE YOUR BROADCAST HERE
    # ============================================================
    
    # Your message
    message = """Hello! 👋

This is a broadcast message from our WhatsApp Business account.

We're excited to share our latest updates with you!

Thank you for being our valued customer.

Best regards,
Your Business Team"""
    
    # ============================================================
    # SEND BROADCAST
    # ============================================================
    
    print("Ready to send broadcast?\n")
    print(f"Contacts: {len(contacts)}")
    print(f"Message preview: {message[:100]}...\n")
    
    if True:
        # Send broadcast (text message)
        results = broadcaster.send_broadcast(contacts, message)
        
        # Save results
        save_results_to_file(results)
        
        # Optionally send template message instead
        # results = broadcaster.send_broadcast(
        #     contacts, 
        #     message, 
        #     use_template=True, 
        #     template_name="hello_world"
        # )
    else:
        print("Broadcast cancelled.")


WhatsApp Business API - Broadcast Script

Verifying credentials...
✓ Credentials verified successfully!
Phone Number: 15556342764


Ready to send broadcast?

Contacts: 2
Message preview: Hello! 👋

This is a broadcast message from our WhatsApp Business account.

We're excited to share ou...


WhatsApp Broadcast Started at 2025-10-23 08:46:11
Total contacts: 2
Message type: Text

[1/2] Sending to ++919821789147... 